In [1]:
!pip install PyMuPDF chromadb sentence-transformers tqdm regex
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 21.1 MB/s e

In [2]:
import fitz
import re
import unicodedata
from tqdm import tqdm

PDF_PATH = "so_tay_2.pdf"
OUTPUT_TXT = "so_tay_clean.txt"
CHUNKS_TXT = "chunks.txt"

# ========== 1️⃣ Làm sạch cơ bản ==========
def normalize_text(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("–", "-")
    text = re.sub(r"(?<=\d),(?=\d)", ".", text)
    text = re.sub(r"[^\n a-zA-ZÀ-Ỹà-ỹ0-9.,:;()–\-+•\[\]]", " ", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s+([,.:;)\]\}])", r"\1", text)
    return text.strip()

# ========== 2️⃣ Ghép bảng thành dòng ngang ==========
def fix_grade_blocks(text: str) -> str:
    """
    Ghép các dòng dạng thang điểm dọc thành 1 dòng ngang liền mạch.
    """
    # Ghép các pattern kiểu “8.5 - 10 A” hoặc “B + 3.5”
    lines = text.splitlines()
    fixed_lines = []
    buffer = ""
    for line in lines:
        line = line.strip()
        if not line:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            continue

        # Nối các dòng có vẻ là bảng điểm
        if re.match(r"^[0-9.,\-\+\sa-zA-Z]+$", line):
            buffer += " " + line
        else:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            fixed_lines.append(line)

    if buffer:
        fixed_lines.append(buffer.strip())

    # Làm gọn lại các dòng bảng
    result = []
    for l in fixed_lines:
        # Ví dụ: "8.5 - 10 A 7.8 - 8.4 B + 7.0 - 7.7 B" → tách bằng regex và nối bằng "; "
        if re.search(r"\d+\.\d+\s*-\s*\d+\.\d+", l):
            parts = re.findall(r"\d+(?:\.\d+)?\s*-\s*\d+(?:\.\d+)?\s*[A-Z\+]*", l)
            if parts:
                result.append("; ".join(p.strip() for p in parts))
            else:
                result.append(l)
        else:
            result.append(l)
    return "\n".join(result)

# ========== 3️⃣ Tách điều / mục ==========
def split_sections(text):
    text = re.sub(r"Điều\s*\n\s*(\d+\.)", r"Điều \1", text, flags=re.IGNORECASE)
    text = re.sub(r"Mục\s*\n\s*(\d+\.)", r"Mục \1", text, flags=re.IGNORECASE)
    sections = re.split(r"(?=(?:\n?Điều\s+\d+\.|\n?Mục\s+\d+\.))", text, flags=re.IGNORECASE)
    sections = [s.strip() for s in sections if len(s.strip()) > 150]
    return sections

# ========== MAIN ==========
if __name__ == "__main__":
    print("🚀 Đang đọc PDF & ghép bảng điểm ngang...")
    doc = fitz.open(PDF_PATH)
    pages = []
    for page in tqdm(doc, desc="📖 Trích xuất"):
        text = page.get_text("text")
        text = normalize_text(text)
        text = fix_grade_blocks(text)
        pages.append(text)
    doc.close()

    full_text = "\n\n".join(pages)
    full_text = re.sub(r"SỔ TAY SINH VIÊN KHÓA\s*\d+", "", full_text)
    full_text = re.sub(r"\[\s*Trang\s*\d+\s*\]", "", full_text)
    full_text = re.sub(r"\n{3,}", "\n\n", full_text)

    with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ Lưu văn bản sạch: {OUTPUT_TXT}")

    chunks = split_sections(full_text)
    with open(CHUNKS_TXT, "w", encoding="utf-8") as f:
        f.write("\n\n".join(chunks))
    print(f"✅ Tổng {len(chunks)} đoạn đã lưu vào {CHUNKS_TXT}")


🚀 Đang đọc PDF & ghép bảng điểm ngang...


📖 Trích xuất: 100%|██████████| 232/232 [00:01<00:00, 170.51it/s]

✅ Lưu văn bản sạch: so_tay_clean.txt
✅ Tổng 197 đoạn đã lưu vào chunks.txt


In [3]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

DB_DIR = "vector_store"
COLLECTION = "so_tay_hcmue"
EMB_MODEL_NAME = "BAAI/bge-m3"
INPUT_TXT = "chunks.txt"

with open(INPUT_TXT, "r", encoding="utf-8") as f:
    docs = [x.strip() for x in f.read().split("\n\n") if len(x.strip()) > 80]

print(f"📘 Tổng {len(docs)} đoạn nội dung.")

client = chromadb.PersistentClient(path=DB_DIR, settings=Settings(allow_reset=True))
try:
    client.delete_collection(COLLECTION)
except:
    pass
col = client.create_collection(COLLECTION)

model = SentenceTransformer(EMB_MODEL_NAME)
print("🚀 Đang tạo embeddings...")

embs = []
for i in tqdm(range(0, len(docs), 32)):
    batch = docs[i:i+32]
    batch_emb = model.encode(batch, normalize_embeddings=True).tolist()
    embs.extend(batch_emb)

col.add(ids=[str(i) for i in range(len(docs))],
        documents=docs,
        embeddings=embs)

print(f"✅ Đã lưu {len(docs)} đoạn vào collection '{COLLECTION}'.")


📘 Tổng 409 đoạn nội dung.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

🚀 Đang tạo embeddings...


100%|██████████| 13/13 [00:34<00:00,  2.68s/it]


✅ Đã lưu 409 đoạn vào collection 'so_tay_hcmue'.


In [4]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import re

DB_DIR = "/content/vector_store"            # hoặc "/content/vector_store" trên Colab
COLLECTION = "so_tay_hcmue"
EMB_MODEL_NAME = "BAAI/bge-m3"

# Reset instance cũ (tránh lỗi "already exists")
import chromadb.api
chromadb.api.client.SharedSystemClient._instance = None

client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
col = client.get_collection(COLLECTION)
model = SentenceTransformer(EMB_MODEL_NAME)

def pretty(text: str) -> str:
    # Ép xuống dòng sau ; : . ) ] } nếu theo sau là + - • –
    text = re.sub(r"([;:.\)\]\}])\s*(?=[\+\-•–])", r"\1\n", text)
    # Đưa bullet về đầu dòng
    text = re.sub(r"\s*([+\-•–])\s+", r"\n\1 ", text)
    # Điều x. đầu dòng
    text = re.sub(r"\s*(Điều\s+\d+\.)", r"\n\1", text, flags=re.IGNORECASE)
    # Gom newline
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

print("=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE ===")
print("Nhập 'exit' để thoát.\n")

while True:
    q = input("📘 Nhập nội dung muốn tra cứu: ").strip()
    if not q:
        continue
    if q.lower() == "exit":
        print("👋 Tạm biệt!")
        break

    q_emb = model.encode(q, normalize_embeddings=True).tolist()
    res = col.query(query_embeddings=[q_emb], n_results=3)

    print("\n🔎 Kết quả tìm thấy:")
    for i, doc in enumerate(res["documents"][0]):
        print(f"\n📄 Đoạn {i+1}:")
        print(pretty(doc))
    print("\n-----------------------------\n")


=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE ===
Nhập 'exit' để thoát.

📘 Nhập nội dung muốn tra cứu: Thời gian vào học

🔎 Kết quả tìm thấy:

📄 Đoạn 1:
37
số giờ giảng đối với một học phần bất kỳ không vượt quá 15 giờ tuần và
4 giờ ngày.
5. Việc phân bổ các học kỳ
a) Đối với hình thức đào tạo chính quy, một năm học có hai học kỳ
chính, mỗi học kỳ chính có ít nhất 15 tuần thực học và 03 tuần thi. Ngoài
hai học kỳ chính, tùy tình hình thực tế, Hiệu trưởng quyết định tổ chức
thêm học kỳ phụ có ít nhất 05 tuần thực học và 01 tuần thi để sinh viên có
điều kiện được học lại, học bù hoặc học vượt.
b) Thời gian hoạt động giảng dạy được thực hiện từ 7g00 đến 17g30
và được chia thành các tiết học. Mỗi tiết học tính bằng 50 phút, được bố
trí cụ thể như sau:
Buổi
Sáng
Chiều
Tiết
1 2 3 4 5 6 7 8 9 10
Từ
7g00 7g50 8g50 9g40 10g40 13g00 13g50 14g50 15g40 16g40
Đến
7g50 8g40 9g40 10g30 11g30 13g50 14g40 15g40 16g30 17g30
c) Đối với hình thức đào tạo vừa làm vừa học, thời gian tổ chức các
học kỳ sẽ được quy đị

KeyboardInterrupt: Interrupted by user

In [5]:
!zip -r /content/download.zip /content/vector_store /content/chunks.txt /content/so_tay_clean.txt

from google.colab import files
files.download('/content/download.zip')

  adding: content/vector_store/ (stored 0%)
  adding: content/vector_store/99c9f340-92f6-4f9b-8d3e-654775bf955f/ (stored 0%)
  adding: content/vector_store/99c9f340-92f6-4f9b-8d3e-654775bf955f/header.bin (deflated 63%)
  adding: content/vector_store/99c9f340-92f6-4f9b-8d3e-654775bf955f/link_lists.bin (stored 0%)
  adding: content/vector_store/99c9f340-92f6-4f9b-8d3e-654775bf955f/length.bin (deflated 34%)
  adding: content/vector_store/99c9f340-92f6-4f9b-8d3e-654775bf955f/data_level0.bin (deflated 100%)
  adding: content/vector_store/chroma.sqlite3 (deflated 47%)
  adding: content/chunks.txt (deflated 79%)
  adding: content/so_tay_clean.txt (deflated 79%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>